In [63]:
! pip install psycopg2
import psycopg2
from psycopg2 import pool
import pandas as pd

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


This article was a helpful primer into the different pooling methods of psycopg2: 

https://aartij1998.medium.com/what-is-connection-pooling-8511b8e51472

In [78]:
pooling = psycopg2.pool.ThreadedConnectionPool(1,100, host='postgresql.test-db01.svc.cluster.local',
                                                      user='user82M',
                                                      password='q2s2gLeojWQFMkBa', 
                                                      database ='sampledb')
class PooledDBConnection:
    """Class to establish pooled connection with threading module"""
    def __init__(self):
    """Initalize self.pool and self.conn"""
        self.pool = None
        self.conn = None
    
    def get(self):
    """Assign connection to self.conn"""
        self.conn = pooling.getconn()
        return self.conn
    
    def put(self):
    """Put away a connection"""
        pooling.putconn(self.conn)
    
class AnomalyDataService:
    """Class to execute SQL queries to the database"""
    def __init__(self):
    """Call get method from the PooledDBClass and create a cursor to execute SQL queries"""
        self.pool = PooledDBConnection()
        self.conn= self.pool.get()
        self.cursor = self.conn.cursor()
    
    def get_all_data(self, get_all_dataSQL):
    """Execute SQL queries to the database"""
        try:
            result= self.cursor.execute(get_all_dataSQL)
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        else:
            return result
           
    def close(self):
    """Close all connections including the one currently in use"""
        self.cursor.close()
        self.pool.put()
        
    def get_dfSQL(self):
    """"Perform a SQL query on a database table and reads it into a pandas dataframe"""
        df = pd.read_sql_query(get_all_dataSQL, self.conn)
        return df

Usage example: Let us select all rows from the anomaly table stored in our database where the column value is equal to 0

In [80]:
query = AnomalyDataService()
get_all_dataSQL = """SELECT * FROM anomaly WHERE groupname = 0 """
query.get_dfSQL()

,sensortimestamp,groupname,sensor_25,sensor_11,sensor_36,sensor_34,machine_status
0,2018-04-10 09:55:00,0,718.7458,40.675090,289.5530,163.4677,0
1,2018-04-10 09:56:00,0,723.9152,41.017250,256.5261,163.6026,0
2,2018-04-10 09:57:00,0,723.8909,41.217490,256.9553,166.5880,0
3,2018-04-10 09:58:00,0,717.3254,41.412200,261.0644,164.9973,0
4,2018-04-10 09:59:00,0,721.9371,41.507660,262.5026,162.6964,0
...,...,...,...,...,...,...,...
3596,2018-04-12 21:51:00,0,655.7155,28.343960,295.7441,249.9119,0
3597,2018-04-12 21:52:00,0,657.5817,28.638560,293.1167,253.6228,0
3598,2018-04-12 21:53:00,0,657.5817,28.347830,293.1167,253.6228,0
3599,2018-04-12 21:54:00,0,654.5527,16.553430,298.8596,250.4138,0


In [81]:
# make sure to close connection to the database so that we're not wasting resources
query.close()